<a href="https://colab.research.google.com/github/cagBRT/Machine-Learning/blob/master/Hierarchical_Clustering_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone -l -s https://github.com/cagBRT/Machine-Learning.git cloned-repo
%cd cloned-repo

Import libraries

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline
import numpy as np
from pandas import read_csv
import scipy.cluster.hierarchy as shc
from sklearn.cluster import AgglomerativeClustering

Get the data and prepare the data<br>
The data is from the pima indians diabetes dataset. <br>
The class is 0 or 1. This is the outcome of the person getting diabetes or not,

In [ ]:
path = "pima_indians_diabetes.csv"
headernames = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
data = read_csv(path, names = headernames)

In [ ]:
data

In [ ]:
#!cat pima_indians_diabetes.csv

Convert all the data to numeric data

In [ ]:
data["preg"] = pd.to_numeric(data[1:]["preg"], downcast="float")
data["plas"] = pd.to_numeric(data[1:]["plas"], downcast="float")
data["pres"] = pd.to_numeric(data[1:]["pres"], downcast="float")
data["skin"] = pd.to_numeric(data[1:]["skin"], downcast="float")
data["test"] = pd.to_numeric(data[1:]["test"], downcast="float")
data["age"] = pd.to_numeric(data[1:]["age"], downcast="float")

data["mass"] = pd.to_numeric(data[1:]["mass"], downcast="float")
data["pedi"] = pd.to_numeric(data[1:]["pedi"], downcast="float")
data["age"] = pd.to_numeric(data[1:]["age"], downcast="float")
data["class"] = pd.to_numeric(data[1:]["class"], downcast="float")

In [ ]:
data

Split the data into features and labels

In [ ]:
array = data.values
X = array[:,0:8]
Y = array[:,8]

In [ ]:
Y.shape

Drop the 0 row.

In [ ]:
data = data.drop([0])

In [ ]:
print(data.shape)
data.head()

Examine the data stats

In [ ]:
data.describe

Pick two features to plot

In [ ]:
patient_data = data["skin"], data["test"]

In [ ]:
plt.figure(figsize = (10, 7))
plt.scatter(patient_data[0],patient_data[1])
plt.xlabel("skin")
plt.ylabel("test")

The dendrograms<br>
Starting from each label at the bottom, you can see a vertical line up to a horizontal line. <br>
The height of that horizontal line tells the distance at which this label was merged into another label or cluster. <br>
You can find that other cluster by following the other vertical line down again. <br>

If you don't encounter another horizontal line, it was just merged with the other label you reach, otherwise it was merged into another cluster that was formed earlier.

Horizontal lines are cluster merges<br>
Vertical lines tell you which clusters/labels were part of merge forming that new cluster<br>
Heights of the horizontal lines tell you about the distance that needed to be "bridged" to form the new cluster

In [ ]:
plt.figure(figsize = (20, 10))
plt.title("Patient Dendograms")
plt.xlabel('sample index')
plt.ylabel('distance')
dend = shc.dendrogram(shc.linkage(data, method = 'ward'))

A huge jump in distance is typically what we're interested in if we want to select for a certain number of clusters. 

In [ ]:
def fancy_dendrogram(*args, **kwargs):
    max_d = kwargs.pop('max_d', None)
    if max_d and 'color_threshold' not in kwargs:
        kwargs['color_threshold'] = max_d
    annotate_above = kwargs.pop('annotate_above', 0)

    ddata = shc.dendrogram(*args, **kwargs)

    if not kwargs.get('no_plot', False):
        plt.title('Hierarchical Clustering Dendrogram (truncated)')
        plt.xlabel('sample index or (cluster size)')
        plt.ylabel('distance')
        for i, d, c in zip(ddata['icoord'], ddata['dcoord'], ddata['color_list']):
            x = 0.5 * sum(i[1:3])
            y = d[1]
            if y > annotate_above:
                plt.plot(x, y, 'o', c=c)
                plt.annotate("%.3g" % y, (x, y), xytext=(0, -5),
                             textcoords='offset points',
                             va='top', ha='center')
        if max_d:
            plt.axhline(y=max_d, c='k')
    return ddata


Set the cutoff for the distance

In [ ]:
# set cut-off to 50
max_d = 2500  # max_d as in max_distance
Z = shc.linkage(data, 'ward')

In [ ]:
fancy_dendrogram(
    Z,
    truncate_mode='lastp',
    p=12,
    leaf_rotation=90.,
    leaf_font_size=12.,
    show_contracted=True,
    annotate_above=10,
    max_d=max_d,  # plot a horizontal cut-off line
)
plt.show()

Make predictions

In [ ]:
cluster = AgglomerativeClustering(n_clusters = 2, affinity = 'euclidean', linkage = 'ward')
cluster.fit_predict(patient_data)

scipy.cluster.hierarchy.fcluster can be used to flatten the dendrogram, obtaining as a result an assignation of the original data points to single clusters.<br>

This assignation mostly depends on a distance threshold t - the maximum inter-cluster distance allowed:

In [ ]:
from scipy.cluster.hierarchy import fcluster
clusters = fcluster(Z, max_d, criterion='distance')
clusters

In [ ]:
plt.figure(figsize = (10, 7))
plt.scatter(patient_data[0], patient_data[1], c = clusters, cmap = 'rainbow')
plt.xlabel("skin")
plt.ylabel("test")

In [ ]:
# Python script for confusion matrix creation.
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

In [ ]:
results = confusion_matrix(Y[1:], clusters)
print ('Confusion Matrix :')
print(results)
print ('Accuracy Score :',accuracy_score(Y[1:], clusters))
print('Classification Report : ')
print (classification_report(Y[1:], clusters))